# Testlipids

> Utility classes for building a membrane system containing just one lipid species, compiling it and minimizing it

 Class **testlipids**(lipids[], membrane, insane, MD paramfile, Martini path,clean up temp path).**execute**(verbose).**report**()
* lipids: list of lipid names
* membrane: Name of the system
* insane script to use (insane.py)
* MD parameter file (test.mdp)
* Martini path to the martini topology files
* Clean up temp path (True/False)
* verbose (True/False): determines whether to print a notification as each stage in the test is completed
* report provide summary report of successful and unsuccessful species

In [22]:
import os #Operating system specific commands
#defaults
defaultpath="/home/richard/projects/hNET_neuronal/"
defaultmdp = os.path.join(defaultpath,"debug/test.mdp")
defaultmartini = os.path.join(defaultpath,"debug/martini.ff")
defaultinsane = os.path.join(defaultpath,"debug/insane2015.py")

#verify defaults exist
if not os.path.isfile(defaultmdp): 
    print("WARNING: default MDP [{}] missing".format(defaultmdp))
if not os.path.isfile(defaultinsane): 
    print("WARNING: default insane [{}] missing".format(defaultinsane))
if not os.path.isdir(defaultmartini): 
    print("WARNING: default martini path [{}] missing".format(defaultmartini))

In [23]:
import os #Operating system specific commands
class build:
    def __init__(self,path,membrane,lipid,insane):
        self.insane=os.path.abspath(insane)
        self.path=path
        self.membrane=membrane
        self.lipid=lipid
        self.success=False
        self.summary =""
    def execute(self):
        #filename for this lipid from this membrane
        membranelipid = os.path.join(self.path,self.membrane+self.lipid)
        NULL = !rm {membranelipid}.gro 
        self.summary = !python2 {self.insane} -o {membranelipid}.gro -p {membranelipid}.top -d 0 -x 27 -y 27 -z 15 -sol PW -salt 0.15 -center -u {self.lipid}:1  -l {self.lipid}:1 
        self.success=os.path.isfile(membranelipid+'.gro')
        return self        
    def report(self,results):
        if not self.success:
            results.append([self.lipid,self.success,self.summary])
        return self

In [24]:
import re #Regular expression library
class compile:
    def __init__(self,path,membrane,lipid,mdfile,martinipath):
        self.path=path
        self.membrane=membrane
        self.lipid=lipid
        self.success=False
        self.summary =""
        self.output=""
        self.mdfile=mdfile 
        self.martinipath=martinipath
    def execute(self):
        membranelipid = os.path.join(self.path,self.membrane+self.lipid)
        
        tempmartini = os.path.join(self.path,self.martinipath)
        !cp -r {self.martinipath} {self.path}
        self.output = !gmx grompp -f {self.mdfile}  -c {membranelipid}.gro -p {membranelipid}.top -o  {membranelipid}.tpr 
        #check for Fatal Error in output and emit subsequent lines to summary
        error = []
        fe = False
        try:
            for line in self.output:
                if fe:
                    error.append(line)
                elif re.search("Fatal error", line):
                    fe = True
                elif re.search("Error in user input", line):
                    fe = True
        except IOError as exc:
            self.success=False
            self.summary=exc 
            results.append([self.lipid,False,exc])
        finally:
            if error:
                self.success=False
                self.summary=" ".join(error)
            else:
                self.success=True
        return self
    def report(self,results):
        if not self.success: 
            results.append([self.lipid,self.success,self.summary])
        return self
    def succeeded(self):
        return self.success;

In [25]:
import os #Operating system specific commands
class minimize:
    def __init__(self,path,membrane,lipid):
        self.path=path
        self.membrane=membrane
        self.lipid=lipid
        self.success=False
        self.summary =""
    def execute(self):
        membranelipid = os.path.join(self.path,self.membrane+self.lipid)
        !export GMX_MAXCONSTRWARN=-1
        !export GMX_SUPPRESS_DUMP=1
        self.output = !gmx mdrun -v -deffnm {membranelipid} 
        logfile = membranelipid+".log"
        if not os.path.exists(logfile):
            self.sucess=False
            self.summary=self.output
        else:
            try:
                file = open(logfile, "r")
                fe = False
                for line in file:
                    if fe:
                        self.success=False
                        self.summary=line
                    elif re.search("^Steepest Descents.*converge", line):
                        self.success=True
                        self.summary=line
                        return self
                    elif re.search("Fatal error", line):
                        fe = True
            except IOError as exc:
                self.sucess=False;
                self.summary=exc;
        return self
    def report(self,results):
        if not self.success:
            results.append([self.lipid,self.success,self.summary])
        return self

In [26]:
import os #Operating system specific commands
from colorama import Fore
import time
import tempfile
import shutil

class testlipids:
    def __init__(self,lipids=None,membrane="",insane=defaultinsane,testmdp=defaultmdp,martinipath=defaultmartini,deleteTMP=True):
        if not lipids:
            print("usage testlipids(lipid list, Membrane name, insane script, test params, martini topology path, delete temporary paths).execute(verbose).report()")
        self.lipids=lipids
        self.membrane=membrane
        self.insane=os.path.abspath(insane)
        self.testmdp = os.path.abspath(testmdp)
        self.martinipath = os.path.abspath(martinipath)
        self.deleteTMP = deleteTMP
        self.results = []    
    def execute(self, verbose):
        count=len(self.lipids)
        startingdir = os.getcwd()
        for lipid in self.lipids:
            start = time.perf_counter()
            if verbose:
                print("[{}]testing lipid {} in membrane {}".format(count,lipid,self.membrane), end='')
            count=count-1
            
            try: 
                tmpdirname = tempfile.mkdtemp()
                os.chdir(tmpdirname)
                
                #filename for this lipid from this membrane
                membranelipid = os.path.join(tmpdirname,self.membrane+lipid)
                # create structure
                if verbose:
                    print('...create', end='')

                if not build(tmpdirname,self.membrane,lipid,self.insane).execute().report(self.results).success:
                    if verbose:
                        print(Fore.RED+'=[fail]'+Fore.BLACK, end='')
                else:
                    #Compile 
                    if verbose:
                        print('...compile', end='')
                    comp = compile(tmpdirname,self.membrane,lipid,self.testmdp, self.martinipath).execute().report(self.results);    
                    if not comp.success:
                        if verbose:
                            print(Fore.RED+'=[fail]'+Fore.BLACK, end='')
                    else:
                        if verbose:
                            print('...minimize', end='')
                        if not minimize(tmpdirname,self.membrane,lipid).execute().report(self.results).success:
                            if verbose:
                                print(Fore.RED+'=[fail]'+Fore.BLACK, end='')
                        else:
                            self.results.append([lipid,True,""])
                            if verbose:
                                print(Fore.GREEN+'=[success]'+Fore.BLACK, end='')
                if verbose:
                    print(": %.2f secs" % (time.perf_counter()-start))        
            
            finally:
                if not self.deleteTMP:
                    if verbose:
                        print('path [{}]'.format(tmpdirname))
                else:
                    shutil.rmtree(tmpdirname)
                os.chdir(startingdir)
                pass
        return self
    def report(self):
        sucesses=[]
        failures=[]
        reasons=[]
        for result in self.results:
            if result[1]:
                sucesses.append(result[0])
            else:
                failures.append(result[0])
                reasons.append("{}: {}".format(result[0],result[2]))
        print("")
        print("Insane:"+self.insane)
        print(Fore.GREEN+("Minimized {} lipids: ".format(len(sucesses)))+Fore.BLACK,end='')
        print(sucesses)
        print(Fore.RED+("Failed to minimize {} lipids: ".format(len(failures)))+Fore.BLACK)
        print(failures)
        print("Summaries of failed minimizations:")
        print(reasons)
        return self

# Classes for querying systems

## extract a list of lipids from a fragment of an insane commandline

call Lipidsfromcmdline() for usage

In [27]:
def lipidsfromcmdline(cmdline=None):
    if not cmdline:
        print("Usage: lipidsfromcmdline(insane commandline fragment)")
        print("example: lipidsfromcmdline(' -u CHOL:0.444 -u POPC:0.087 ')")
    else:
        lipids=[]
        cmds=cmdline.split()
        for cmd in cmds:
            if ":" in cmd:
                lipids.append(cmd.split(":")[0])
    return lipids

## extract a list of lipids from a .gro or .pdb file

call Lipidsfromsystem() for usage

In [28]:
import os #Operating system specific commands

#Exeception class for an imediate stop
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

def lipidsfromsystem(systemfilename=None):
    if not systemfilename:
        print("Usage: lipidsfromsystem(.gro or .pdb filename)")
        print("example: lipidsfromsystem(hNET_neuronal)")
        print(" If a .gro file exists that will be first converted into a .pdb")
    else:
        # Generate list of lipid names from inputs
        if os.path.isfile(systemfilename+'.gro'): # If the input is a .gro file ... convert it first into a pdb file
            print("Converting "+systemfilename+".gro to a pdb file")
            output = !gmx editconf -f {systemfilename}.gro -o {systemfilename}.pdb
        if os.path.isfile(systemfilename+'.pdb'): # If the input is a .pdb file ... extract all species not amino acids or solvent
            print("using system specified in "+systemfilename+".pdb file")
            pdbfile = open(systemfilename+".pdb")
            lipids=[]
            AAs=["ASP","GLY","ALA","GLN","PRO","ARG","GLU","THR","TRP","LYS","ILE","PHE","SER","VAL","TYR","CYS","LEU","MET","ASN","HIS",]
            solvent=["PW","CL-","NA+","ION"]
            for line in pdbfile:
                col = line.split() 
                if len(col)>1:
                    atom= col[0]
                    if atom.strip()=="ATOM":
                        segment=col[3]
                        if (segment not in AAs) and (segment not in solvent):
                            if segment not in lipids: 
                                lipids.append(segment)
        if not lipids:
            print('No lipids specified')
            raise StopExecution
        else:
            return lipids

    

### Class to query a .gro file to display all lipids that match lipids in array

Usage: gro(tmpdirname,membrane,lipid[]).analyze().reportlipid()

In [29]:
# class for querying a gro file
import os
import re
from collections import defaultdict
class gro:
    def __init__(self,path,membrane,lipid):
        self.path=path
        self.membrane=membrane
        self.lipid=lipid
        self.file = open(os.path.join(self.path,self.membrane+self.lipid)+".gro","r")
        self.species=defaultdict(int)
        self.sample=[]
    def __del__(self):
        self.file.close;
    def analyze(self):
        #load all rows with 6 split columns
        self.file.readline()
        self.file.readline()
        samplemolecule=None
        for line in self.file:
            molecule = line[1:9].strip() 
            start=0
            while start < len(molecule) and molecule[start] in "0123456789.":
                start=start+1
            moleculename = molecule[start:len(molecule)]
            if moleculename:
                self.species[moleculename]+= 1
            if not samplemolecule:
                if moleculename==self.lipid:
                    samplemolecule=molecule
            if molecule==samplemolecule:
                self.sample.append([molecule,line])
        return self
    def reportlipid(self):
        for line in self.sample:
            print(line)
        return self
        

In [30]:
import os
class top:
    def __init__(self,path,membrane,lipid):
        self.path=path
        self.membrane=membrane
        self.lipid=lipid
        self.file = open(os.path.join(self.path,self.membrane+self.lipid)+".top","r")
    def __del__(self):
        self.file.close;
    def analyze(self):
        for line in self.file:
            print(line)
        return self


# Tests 

> NB: these will only run when the notebook is loaded directly and run, they will not run when the notebook is loaded as a %run module from other notebooks

## Test: testlipds

Construct a membrane with CHOL and PAP3, using the version of insane that incorrectly builds PAP3, test each lipid in the system to minimization, and report errors in verbose mode, and do not clean up temp files so we can inspect files created

In [32]:
# the following code will not  run if the notebook is loaded as a %run library
if __name__ == '__main__' and '__file__' not in globals():
    testlipids(["PAP3","CHOL"],"hNET_neuronal","../insane.py","./test.mdp",'./martini.ff',False).execute(True).report()

[2]testing lipid PAP3 in membrane hNET_neuronal...create...compile...minimize=[fail]: 5.77 secs
path [/tmp/tmpzjne70x2]
[1]testing lipid CHOL in membrane hNET_neuronal...create...compile...minimize=[success]: 34.43 secs
path [/tmp/tmpwbxs0mo8]

Insane:/home/richard/projects/hNET_neuronal/insane.py
Minimized 1 lipids: ['CHOL']
Failed to minimize 1 lipids: 
['PAP3']
Summaries of failed minimizations:
['PAP3: -------------------------------------------------------\n']


## Test: testlipids() usage display

In [34]:
# the following code will not  run if the notebook is loaded as a %run library
if __name__ == '__main__' and '__file__' not in globals():
    testlipids()

usage testlipids(lipid list, Membrane name, insane script, test params, martini topology path, delete temporary paths).execute(verbose).report()


## Test: testlipds

Construct a membrane with CHOL and POPC, using the default version of insane, test each lipid in the system to minimization, and report errors in verbose mode, and clean up temp files

In [35]:
# the following code will not  run if the notebook is loaded as a %run library
if __name__ == '__main__' and '__file__' not in globals():
    testlipids(lipidsfromcmdline("-u CHOL:0.444 -u POPC:0.087")).execute(True).report()

[2]testing lipid CHOL in membrane ...create...compile...minimize=[success]: 32.34 secs
[1]testing lipid POPC in membrane ...create...compile...minimize=[success]: 13.70 secs

Insane:/home/richard/projects/hNET_neuronal/debug/insane2015.py
Minimized 2 lipids: ['CHOL', 'POPC']
Failed to minimize 0 lipids: 
[]
Summaries of failed minimizations:
[]


## Test: testlipds from a system

Extract lipids from a broken system, and create a membrane created for each species, using the default version of insane, test each lipid in the system to minimization, and report errors in verbose mode, and clean up temp files

In [20]:
# the following code will not  run if the notebook is loaded as a %run library
if __name__ == '__main__' and '__file__' not in globals():
    lipids=lipidsfromsystem("/home/richard/Desktop/projects/hNET_neuronal/debug/hNET_neuronal")
    print(lipids)
    testlipids(lipids).execute(True).report()

Converting /home/richard/Desktop/projects/hNET_neuronal/debug/hNET_neuronal.gro to a pdb file
using system specified in /home/richard/Desktop/projects/hNET_neuronal/debug/hNET_neuronal.pdb file
['CHOL', 'POPC', 'DPSM', 'DPPC', 'PUPE', 'DPGS', 'PAPC', 'PAPE', 'DOPC', 'PUPC', 'POPE', 'PNSM', 'PBSM', 'PNGS', 'DPG1', 'DPG3', 'DBGS', 'OAPE', 'OUPE', 'POSM', 'PFPC', 'OIPC', 'POGS', 'OUPC', 'DPCE', 'PADG', 'DBG1', 'PNG1', 'DBG3', 'PNG3', 'PPC', 'OIPE', 'POG1', 'POG3', 'DBCE', 'PNCE', 'IPC', 'PPE', 'IPE', 'PODG', 'PUPS', 'PAPS', 'POPS', 'PUPI', 'POPI', 'PAPI', 'OUPS', 'DPPS', 'PIPI', 'PAPA', 'POP1', 'POP2', 'POP3', 'POPA']
[54]testing lipid CHOL in membrane ...create...compile...minimize=[success]: 12.26 secs
[53]testing lipid POPC in membrane ...create...compile...minimize=[success]: 5.53 secs
[52]testing lipid DPSM in membrane ...create...compile...minimize=[success]: 6.48 secs
[51]testing lipid DPPC in membrane ...create...compile...minimize=[success]: 5.64 secs
[50]testing lipid PUPE in me

## Test: Query .top file for list of all species

Create a temporary path. Build a membrane containing just one lipid species (PNG1), then view the top file created, then clean up temporary path

In [19]:
# the following code will not  run if the notebook is loaded as a %run library
if __name__ == '__main__' and '__file__' not in globals():
    membrane="hNET_neuronal"
    lipid="PNG1"
    with tempfile.TemporaryDirectory() as tmpdirname:
        membranelipid = os.path.join(tmpdirname,membrane+lipid)
        results = []
        build(tmpdirname,membrane,lipid,defaultinsane).execute().report(results).success
        top(tmpdirname,membrane,lipid).analyze()

#include "martini.ff/martini_v2.2P.itp"

#include "martini.ff/martini_v2.0_ions.itp"

#include "martini.ff/martini_v2.0_lipids_all_201506.itp"



[ system ]

; name

INSANE! Membrane UpperLeaflet>PNG1=1.0 LowerLeaflet>PNG1=1.0



[ molecules ]

; name  number

PNG1          1225

PNG1          1225

PW           35520

NA+            391

CL-            391



## Test: query a gro file for all rows for a specific lipid


In [12]:
import tempfile
import os
# the following code will not  run if the notebook is loaded as a %run library
if __name__ == '__main__' and '__file__' not in globals():
    membrane="hNET_neuronal"
    lipid="PAP3"
    with tempfile.TemporaryDirectory() as tmpdirname:
        membranelipid = os.path.join(tmpdirname,membrane+lipid)
        results = []
        print(build(tmpdirname,membrane,lipid,'/home/richard/projects/hNET_neuronal/insane.py').execute().report(results).summary)
        gro(tmpdirname,membrane,lipid).analyze().reportlipid()
        

['; X: 27.000 (35 lipids) Y: 27.000 (35 lipids)', '; 1225 lipids in upper leaflet, 1225 lipids in lower leaflet', '; NDX Solute 1 0', '; Charge of protein: 0.000000', '; NDX Membrane 1 44100', '; Charge of membrane: 0.000000', '; Total charge: 0.000000', '; NDX Solvent 44101 224669', '; NDX System 1 224669']
['1PAP3', '    1PAP3    C1    1  20.455  20.502   9.450\n']
['1PAP3', '    1PAP3    C2    2  20.346  20.589   9.750\n']
['1PAP3', '    1PAP3    C3    3  20.481  20.444   9.750\n']
['1PAP3', '    1PAP3   PO4    4  20.523  20.458  10.050\n']
['1PAP3', '    1PAP3    P1    5  20.375  20.520  10.350\n']
['1PAP3', '    1PAP3    P2    6  20.495  20.466  10.350\n']
['1PAP3', '    1PAP3    P3    7  20.459  20.493  10.650\n']
['1PAP3', '    1PAP3   GL1    8  20.474  20.541   9.150\n']
['1PAP3', '    1PAP3   GL2    9  20.442  20.583   9.150\n']
['1PAP3', '    1PAP3   D1A   10  20.533  20.473   8.850\n']
['1PAP3', '    1PAP3   D2A   11  20.537  20.509   8.550\n']
['1PAP3', '    1PAP3   D3A   1